In [ ]:
# import packages and tools

In [ ]:
# import needed data

In [ ]:
# load in csvs for each cluster desired into dataframes

In [ ]:
# create county index hashtable

In [ ]:
# convert each dataframe into np array

In [ ]:
# run k means clustering for each np array